# Investigation Template
This template contains all inputs required for any sort of investigation. That way there should be fields that you won't need to fill, so please leave them blank. Feel free to send me feedback so that I can update the notebook. Thanks!
Run every cell first before trying to generate the report!

P.S. Don't forget to add your API keys if you want to check hashes/ips/links reputation. 

## API Keys

In [3]:
# CODE
import ipywidgets as widgets
from IPython.display import display, clear_output

# Input fields
vt_api_input = widgets.Password(placeholder='Enter VirusTotal API key', description='VT API:')
abuse_api_input = widgets.Password(placeholder='Enter AbuseIPDB API key', description='AbuseIPDB:')
urlscan_api_input = widgets.Password(placeholder='Enter URLScan.io API key', description='URLScan:')
scam_api_input = widgets.Password(placeholder='Enter Scamalytics API key', description='Scamalytics API key:')
scam_host_input = widgets.Password(placeholder='Enter Scamalytics Account Name', description='Scamalytics Account Name:')

# Button and status output
save_btn = widgets.Button(description='Save API Keys', button_style='success')
save_status = widgets.Output()

# Dictionary to store keys for later use
api_keys = {}

def save_keys(_):
    api_keys['VT'] = vt_api_input.value.strip()
    api_keys['AbuseIPDB'] = abuse_api_input.value.strip()
    api_keys['URLScan'] = urlscan_api_input.value.strip()
    api_keys['ScamKey'] = scam_api_input.value.strip()
    api_keys['ScamName'] = scam_host_input.value.strip()
    with save_status:
        clear_output()
        print("✅ API keys ready to use.")

save_btn.on_click(save_keys)

# Display the key input cell
display(widgets.VBox([
    vt_api_input,
    abuse_api_input,
    urlscan_api_input,
    scam_host_input,
    scam_api_input,
    save_btn,
    save_status
]))

## Triage

In [78]:
# CODE
import ipywidgets as widgets
from IPython.display import display, Markdown

# Store all duplicates in a list
duplicate_event_ids = []

# Input field
event_input = widgets.Text(placeholder="[Splunk Event_Id]")
add_btn = widgets.Button(description="Add Duplicate", button_style="info")
event_display = widgets.Output()

def add_event_id(_):
    if event_input.value.strip():
        duplicate_event_ids.append(event_input.value.strip())
        event_input.value = ""
        update_event_display()
    else:
        pass 

def update_event_display():
    event_display.clear_output()
    with event_display:
        if duplicate_event_ids:
            for idx, eid in enumerate(duplicate_event_ids, 1):
                print(f"{idx}. event_id={eid}")
        else:
            print("No duplicates found.")

add_btn.on_click(add_event_id)

display(widgets.VBox([
    widgets.HBox([event_input, add_btn]),
    event_display
]))

def get_triage_section():
    if not duplicate_event_ids:
        return "[+] ------------------------- TRIAGE ------------------------- [+]\nNo duplicates found.\n"
    return "[+] ------------------------- TRIAGE ------------------------- [+]\n" + "\n".join(f"event_id={eid}" for eid in duplicate_event_ids) + "\n"

## Entities

In [77]:
# CODE
import ipywidgets as widgets
from IPython.display import display

# Storage for user and host data
users = []
hosts = []

def create_user_form():
    user_widgets = {
        "user": widgets.Text(placeholder="[User]", layout=widgets.Layout(width='100%')),
        "job_title": widgets.Text(placeholder="[Job Title]", layout=widgets.Layout(width='100%')),
        "company": widgets.Text(placeholder="[Company]", layout=widgets.Layout(width='100%')),
        "entra_logs": widgets.Text(placeholder="[Entra ID sign-in logs]", layout=widgets.Layout(width='100%')),
        "entraaudit_logs": widgets.Text(placeholder="[Entra ID audit logs]", layout=widgets.Layout(width='100%'))
    }

    remove_button = widgets.Button(description="Remove User", button_style='danger')
    user_form = widgets.VBox([
        widgets.HBox([widgets.Label("User Details"), remove_button], layout=widgets.Layout(justify_content='space-between')),
        user_widgets["user"],
        user_widgets["job_title"],
        user_widgets["company"],
        user_widgets["entra_logs"],
        user_widgets["entraaudit_logs"]
    ])

    return user_widgets, user_form, remove_button

def create_host_form():
    host_widgets = {
        "hostname": widgets.Text(placeholder="[Hostname]", layout=widgets.Layout(width='100%')),
        "ipaddress": widgets.Text(placeholder="[Local Ip Address]", layout=widgets.Layout(width='100%')),
        "os": widgets.Text(placeholder="[OS]", layout=widgets.Layout(width='100%'))
    }

    remove_button = widgets.Button(description="Remove Host", button_style='danger')
    host_form = widgets.VBox([
        widgets.HBox([widgets.Label("Host Details"), remove_button], layout=widgets.Layout(justify_content='space-between')),
        host_widgets["hostname"],
        host_widgets["ipaddress"],
        host_widgets["os"]
    ])

    return host_widgets, host_form, remove_button

user_containers = widgets.VBox()
host_containers = widgets.VBox()

def add_user(_):
    user_widgets, user_form, remove_button = create_user_form()
    accordion = widgets.Accordion(children=[user_form])
    accordion.set_title(0, "User " + str(len(user_containers.children) + 1))

    def remove_user(_):
        user_containers.children = tuple(c for c in user_containers.children if c is not accordion)
        users.remove({"widgets": user_widgets, "accordion": accordion})

    remove_button.on_click(remove_user)

    user_containers.children += (accordion,)
    users.append({"widgets": user_widgets, "accordion": accordion})

def add_host(_):
    host_widgets, host_form, remove_button = create_host_form()
    accordion = widgets.Accordion(children=[host_form])
    accordion.set_title(0, "Host " + str(len(host_containers.children) + 1))

    def remove_host(_):
        host_containers.children = tuple(c for c in host_containers.children if c is not accordion)
        hosts.remove({"widgets": host_widgets, "accordion": accordion})

    remove_button.on_click(remove_host)

    host_containers.children += (accordion,)
    hosts.append({"widgets": host_widgets, "accordion": accordion})

add_user_button = widgets.Button(description="Add User", button_style='info')
add_host_button = widgets.Button(description="Add Host", button_style='info')

add_user_button.on_click(add_user)
add_host_button.on_click(add_host)

# Display initial setup
display(add_user_button, add_host_button, user_containers, host_containers)

# Add initial user and host
add_user(None)
add_host(None)

def get_entities_section():
    lines = []

    # Process Users
    user_lines = []
    for user in users:
        user_widgets = user["widgets"]
        current_user_lines = []

        if user_widgets["user"].value.strip():
            current_user_lines.append(f"User: {user_widgets['user'].value.strip()}")
        if user_widgets["job_title"].value.strip():
            current_user_lines.append(f"Job Title: {user_widgets['job_title'].value.strip()}")
        if user_widgets["company"].value.strip():
            current_user_lines.append(f"Company: {user_widgets['company'].value.strip()}")
        if user_widgets["entra_logs"].value.strip():
            current_user_lines.append(f"Entra ID Sign-in Logs: {user_widgets['entra_logs'].value.strip()}")
        if user_widgets["entraaudit_logs"].value.strip():
            current_user_lines.append(f"Entra ID Audit Logs: {user_widgets['entraaudit_logs'].value.strip()}")

        if current_user_lines:
            user_lines.extend(current_user_lines)
            user_lines.append("---")  # Separator between users

    # Remove the trailing separator for users if there are any
    if user_lines:
        lines.extend(user_lines[:-1])  # Add all user lines except the last separator

    # Process Hosts
    host_lines = []
    for host in hosts:
        host_widgets = host["widgets"]
        current_host_lines = []

        if host_widgets["hostname"].value.strip():
            current_host_lines.append(f"Hostname: {host_widgets['hostname'].value.strip()}")
        if host_widgets["ipaddress"].value.strip():
            current_host_lines.append(f"Local IP Address: {host_widgets['ipaddress'].value.strip()}")
        if host_widgets["os"].value.strip():
            current_host_lines.append(f"OS: {host_widgets['os'].value.strip()}")

        if current_host_lines:
            host_lines.extend(current_host_lines)
            host_lines.append("---")  # Separator between hosts

    # If both user and host lines exist, add a separator between them
    if user_lines and host_lines:
        lines.append("---")

    # Add host lines, removing the trailing separator if there are any hosts
    if host_lines:
        lines.extend(host_lines[:-1])

    if not lines:
        return ""

    return "[+] ------------------------ ENTITIES ------------------------ [+]\n" + "\n".join(lines) + "\n"


Button(button_style='info', description='Add User', style=ButtonStyle())

Button(button_style='info', description='Add Host', style=ButtonStyle())

VBox()

VBox()

## Tanium Investigation

In [76]:
# CODE
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests

# --- Fetch VirusTotal Hash Results ---
def get_api_key(source):
    return api_keys.get(source, "")

def fetch_vt_for_hash(sha256):
    vt_key = get_api_key("VT")
    if not vt_key:
        return "No VT API key"
    try:
        r = requests.get(f"https://www.virustotal.com/api/v3/files/{sha256}", headers={"x-apikey": vt_key})
        if r.status_code == 200:
            json_data = r.json()
            malicious = json_data.get('data', {}).get('attributes', {}).get('last_analysis_stats', {}).get('malicious', 0)
            return f"{malicious} malicious detections"
        else:
            return f"VT API returned status {r.status_code}"
    except Exception as e:
        return f"Error retrieving VT: {e}"

# --- Intel block creation function ---
intel_event_blocks = []
container = widgets.VBox()

def create_host_accordion():
    hostname = widgets.Text(placeholder="Hostname", layout=widgets.Layout(width="200px"))
    action = widgets.Text(placeholder="Action Taken", layout=widgets.Layout(width="200px"))
    timestamp = widgets.Text(placeholder="Timestamp", layout=widgets.Layout(width="250px"))
    host_header = widgets.HBox([hostname, action, timestamp])

    # Processes Parameters
    child_process = widgets.Text(placeholder="Child Process", layout=widgets.Layout(width="100%"))
    child_sha256 = widgets.Text(placeholder="Child SHA256", layout=widgets.Layout(width="100%"))
    child_vt_result = widgets.Label()
    check_child_btn = widgets.Button(description="Check Child Hash", button_style='info')
    check_child_btn.on_click(lambda b: child_vt_result.__setattr__('value', fetch_vt_for_hash(child_sha256.value.strip())) if child_sha256.value else "No hash")

    parent_process = widgets.Text(placeholder="Parent Process", layout=widgets.Layout(width="100%"))
    parent_sha256 = widgets.Text(placeholder="Parent SHA256", layout=widgets.Layout(width="100%"))
    parent_vt_result = widgets.Label()
    check_parent_btn = widgets.Button(description="Check Parent Hash", button_style='info')
    check_parent_btn.on_click(lambda b: parent_vt_result.__setattr__('value', fetch_vt_for_hash(parent_sha256.value.strip())) if parent_sha256.value else "No hash")

    process_tree = widgets.Textarea(placeholder="Process Tree", layout=widgets.Layout(width="100%", height="60px"))

    # Process Tree formatter
    format_btn = widgets.Button(description="Format Process Tree", button_style='info')
    format_output = widgets.Output()

    def format_process_tree(b):
        with format_output:
            clear_output()
            raw = process_tree.value.strip().replace('\r\n', '\n').split('\n')
            if len(raw) % 4 != 0:
                print(f"[!] Invalid input: Expected multiples of 4 lines per process, but got {len(raw)} lines.")
                return

            # Group in chunks of 4
            chunks = [raw[i:i+4] for i in range(0, len(raw), 4)]
            chunks.reverse()  # earliest process first

            formatted = ""
            for i, chunk in enumerate(chunks):
                if len(chunk) < 4:
                    print("[!] Skipping incomplete chunk:", chunk)
                    continue
                proc_name, user, pid, full_cmd = chunk
                indent = " " * i
                arrow = "↳ " if i > 0 else ""
                formatted += f'{indent}{arrow}{full_cmd} ({user}, PID:{pid})\n'

            process_tree.value = formatted.strip()

    format_btn.on_click(format_process_tree)
    
    file_activity = widgets.Textarea(placeholder="File Activity", layout=widgets.Layout(width="100%", height="60px"))
    registry_changes = widgets.Textarea(placeholder="Registry Changes", layout=widgets.Layout(width="100%", height="60px"))
    network_activity = widgets.Textarea(placeholder="Network Activity", layout=widgets.Layout(width="100%", height="60px"))
    dns_calls = widgets.Textarea(placeholder="DNS Calls", layout=widgets.Layout(width="100%", height="60px"))
    additional_info = widgets.Textarea(placeholder="Additional Info", layout=widgets.Layout(width="100%", height="60px"))

    # Display
    content = widgets.VBox([
        widgets.HTML("<b>Process Information</b>"),
        child_process, widgets.HBox([child_sha256, check_child_btn]), child_vt_result,
        parent_process, widgets.HBox([parent_sha256, check_parent_btn]), parent_vt_result,
        widgets.HTML("<b>Investigation Details</b>"),
        process_tree, format_btn, format_output, file_activity, registry_changes, network_activity, dns_calls, additional_info
    ])

    # Collapse code
    accordion = widgets.Accordion(children=[widgets.VBox([host_header, content])])
    accordion.set_title(0, hostname.placeholder or "Host Info")

    accordion.metadata = {
        "hostname": hostname, "action": action, "timestamp": timestamp,
        "child_process": child_process, "child_sha256": child_sha256, "child_vt_result": child_vt_result,
        "parent_process": parent_process, "parent_sha256": parent_sha256, "parent_vt_result": parent_vt_result,
        "process_tree": process_tree, "file_activity": file_activity, "registry_changes": registry_changes,
        "network_activity": network_activity, "dns_calls": dns_calls, "additional_info": additional_info
    }

    return accordion

# Intel and Hostname addition button
def create_intel_event_block():
    intel_event = widgets.Text(placeholder="Intel Event", layout=widgets.Layout(width="500px"))
    remove_button = widgets.Button(description="Remove Event", button_style="danger")
    hosts_box = widgets.VBox()

    def add_host(_=None):
        new_host = create_host_accordion()
        hosts_box.children += (new_host,)

    add_host_button = widgets.Button(description="Add Host", button_style="info")
    add_host_button.on_click(add_host)

    block = widgets.VBox()
    accordion = widgets.Accordion(children=[block])
    accordion.set_title(0, f"Intel Event {len(container.children) + 1}")

    def remove_event(_):
        intel_event_blocks.remove(block)
        container.children = tuple(c for c in container.children if c is not accordion)

    remove_button.on_click(remove_event)

    block.children = [
        widgets.HTML("<b>Intel Event</b>"),
        intel_event,
        remove_button,
        widgets.HTML("<b>Affected Hosts</b>"),
        hosts_box,
        add_host_button
    ]

    # 1 host as default
    add_host()

    block.metadata = {
        "intel_event": intel_event,
        "hosts_box": hosts_box
    }

    return accordion, block

def on_add_event(_):
    accordion, block = create_intel_event_block()
    intel_event_blocks.append(block)
    container.children += (accordion,)

add_event_button = widgets.Button(description="Add Intel Event", button_style="success")
add_event_button.on_click(on_add_event)

display(widgets.VBox([
    widgets.HTML("<h4>Tanium Intel Events</h4>"),
    add_event_button,
    container
]))

def get_tanium_section():
    if not intel_event_blocks:
        return ""

    section_lines = []

    for block in intel_event_blocks:
        meta = block.metadata
        intel_val = meta['intel_event'].value.strip()

        if intel_val:
            section_lines.append(intel_val)
            for host in meta['hosts_box'].children:
                m = host.metadata
                h, a, t = m['hostname'].value.strip(), m['action'].value.strip(), m['timestamp'].value.strip()
                if h:
                    host_info = f"- Hostname: {h}"
                    if a: host_info += f" | Action: {a}"
                    if t: host_info += f" | Timestamp: {t}"
                    section_lines.append(host_info)

                # Child Process
                child_process_info = []
                if m["child_process"].value or m["child_sha256"].value:
                    child_process_info.append(f"- Child Process: {m['child_process'].value}")
                    if m["child_sha256"].value:
                        child_process_info.append(f" | SHA256: {m['child_sha256'].value}")
                        if m["child_vt_result"].value:
                            child_process_info.append(f" | VT: {m['child_vt_result'].value}")
                    section_lines.append(''.join(child_process_info))

                # Parent Process
                parent_process_info = []
                if m["parent_process"].value or m["parent_sha256"].value:
                    parent_process_info.append(f"- Parent Process: {m['parent_process'].value}")
                    if m["parent_sha256"].value:
                        parent_process_info.append(f" | SHA256: {m['parent_sha256'].value}")
                        if m["parent_vt_result"].value:
                            parent_process_info.append(f" | VT: {m['parent_vt_result'].value}")
                    section_lines.append(''.join(parent_process_info))

                # Investigation details
                for label, widget_key in [
                    ("Process Tree", "process_tree"),
                    ("File Activity", "file_activity"),
                    ("Registry Changes", "registry_changes"),
                    ("Network Activity", "network_activity"),
                    ("DNS Calls", "dns_calls"),
                    ("Additional Info", "additional_info")
                ]:
                    value = m[widget_key].value.strip()
                    if value:
                        section_lines.append(f"{label}:\n{value}")

            # Add separator after each intel event
            section_lines.append("---")

    # Remove trailing separator if it exists
    if section_lines and section_lines[-1] == "---":
        section_lines = section_lines[:-1]

    if not section_lines:
        return ""

    return "[+] ------------------ TANIUM INVESTIGATION ------------------ [+]\n" + "\n".join(section_lines) + "\n"


## MDE Investigation

In [75]:
# CODE
import ipywidgets as widgets
from IPython.display import display

mde_blocks = []

def create_activity_block(activity_index, parent_container):
    # Inputs for link and activity
    link_input = widgets.Text(
        placeholder='Activity Link',
        description='Link:',
        layout=widgets.Layout(width='100%'),
        style={'description_width': 'initial'}
    )
    activity_input = widgets.Textarea(
        placeholder='Describe the activity...',
        layout=widgets.Layout(width='100%', height='80px'),
        style={'description_width': 'initial'}
    )

    remove_btn = widgets.Button(
        description='Remove Activity',
        button_style='danger',
        layout=widgets.Layout(width='150px')
    )

    # VBox to hold fields
    activity_form = widgets.VBox([link_input, activity_input, remove_btn])
    activity_accordion = widgets.Accordion(children=[activity_form])
    activity_accordion.set_title(0, f"Activity {activity_index + 1}")

    # Add custom attributes
    activity_accordion.link_input = link_input
    activity_accordion.activity_input = activity_input

    # Remove handler
    def on_remove(b):
        parent_container.children = [child for child in parent_container.children if child != activity_accordion]
        refresh_activity_titles(parent_container)

    remove_btn.on_click(on_remove)

    return activity_accordion

def refresh_activity_titles(container):
    for i, block in enumerate(container.children):
        block.set_title(0, f"Activity {i + 1}")

def create_mde_block(index):
    signature_input = widgets.Text(
        placeholder='Detection Signature',
        description='Signature:',
        layout=widgets.Layout(width='100%'),
        style={'description_width': 'initial'}
    )

    # Activity container and button
    activity_container = widgets.VBox()
    
    def add_activity(b=None):
        new_activity = create_activity_block(len(activity_container.children), activity_container)
        activity_container.children += (new_activity,)

    add_activity_btn = widgets.Button(
        description='Add Activity',
        button_style='info',
        layout=widgets.Layout(width='150px')
    )
    add_activity_btn.on_click(add_activity)

    # Add first activity by default
    add_activity()

    # Remove signature block
    remove_signature_btn = widgets.Button(
        description='Remove Signature',
        button_style='danger',
        layout=widgets.Layout(width='150px')
    )

    def on_remove_signature(b):
        mde_blocks.remove(mde_block)
        refresh_mde_container()

    remove_signature_btn.on_click(on_remove_signature)

    form = widgets.VBox([
        signature_input,
        widgets.Label("Activities:"),
        activity_container,
        add_activity_btn,
        remove_signature_btn
    ])

    mde_block = widgets.Accordion(children=[form])
    mde_block.set_title(0, f"Signature {index + 1}")

    # Store attributes
    mde_block.signature_input = signature_input
    mde_block.activity_container = activity_container

    return mde_block

def refresh_mde_container():
    for i, block in enumerate(mde_blocks):
        block.set_title(0, f"Signature {i + 1}")
    mde_container.children = mde_blocks

# Button to add new signature
add_signature_btn = widgets.Button(description="Add Signature", button_style='success')
def on_add_signature(b=None):
    new_block = create_mde_block(len(mde_blocks))
    mde_blocks.append(new_block)
    refresh_mde_container()

add_signature_btn.on_click(on_add_signature)

# Display container
mde_container = widgets.VBox()
on_add_signature()  # Start with one

mde_main = widgets.Accordion(children=[widgets.VBox([mde_container, add_signature_btn])])
mde_main.set_title(0, "MDE Investigations")
display(mde_main)

# Final report generation
def get_mde_section():
    report_lines = []
    section = "[+] ----------------- DEFENDER INVESTIGATION ----------------- [+]"

    valid_blocks = [b for b in mde_blocks if b.signature_input.value.strip()]
    for i, block in enumerate(valid_blocks):
        sig = block.signature_input.value.strip()
        if not sig:
            continue
        report_lines.append(f"Signature: {sig}\n")
        for activity_block in block.activity_container.children:
            link = activity_block.link_input.value.strip()
            activity = activity_block.activity_input.value.strip()
            if link:
                report_lines.append(f"Link: {link}")
            if activity:
                report_lines.append(f"Activity:\n{activity}")
        if i < len(valid_blocks) - 1:
            report_lines.append("---")

    # Join with a single newline between items
    return section + "\n" + "\n".join(report_lines) + "\n" if report_lines else ""

Accordion(children=(VBox(children=(VBox(children=(Accordion(children=(VBox(children=(Text(value='', descriptio…

## C2/Bad Rep domain activity

In [74]:
# CODE
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import requests
import base64

# --- Your existing defang, get_api_key, api_lookup functions here ---

# New C2 Investigation widgets
c2_domains = widgets.Textarea(
    placeholder="[Destination domains (one per line)]", 
    layout=widgets.Layout(width='100%', height='80px')
)
c2_ips = widgets.Textarea(
    placeholder="[Destination IPs (one per line)]", 
    layout=widgets.Layout(width='100%', height='80px')
)
c2_count = widgets.Text(
    placeholder="[Count]", 
    layout=widgets.Layout(width='50%')
)
c2_user_identity = widgets.Text(
    placeholder="[User Identity]", 
    layout=widgets.Layout(width='50%')
)

# Display the widgets for input
display(widgets.VBox([
    c2_domains,
    c2_ips,
    widgets.HBox([c2_count, c2_user_identity])
]))

# --- Helper function for C2 lookups ---
def api_lookup_c2(value, type_):
    results = []
    # We want the same lookups as Phishing but without URLScan (so VT and AbuseIPDB and Scamalytics)
    vt_key = get_api_key('VT')
    abuse_key = get_api_key('AbuseIPDB')
    scam_key = get_api_key('ScamKey')
    scam_name = get_api_key('ScamName')

    # VirusTotal
    if vt_key:
        try:
            if type_ == "ip":
                r = requests.get(f"https://www.virustotal.com/api/v3/ip_addresses/{value}",
                                 headers={"x-apikey": vt_key})
                if r.status_code == 200:
                    json = r.json()
                    malicious = json['data']['attributes']['last_analysis_stats']['malicious']
                    results.append(f"(VT results: {malicious} malicious detections)")
            elif type_ == "domain":
                # VirusTotal domain lookup
                r = requests.get(f"https://www.virustotal.com/api/v3/domains/{value}",
                                 headers={"x-apikey": vt_key})
                if r.status_code == 200:
                    json = r.json()
                    malicious = json['data']['attributes']['last_analysis_stats']['malicious']
                    results.append(f"(VT results: {malicious} malicious detections)")
        except Exception as e:
            results.append(f"(VT lookup error: {e})")
        time.sleep(15)

    # AbuseIPDB only for IPs
    if type_ == "ip" and abuse_key:
        try:
            r = requests.get(
                "https://api.abuseipdb.com/api/v2/check",
                params={"ipAddress": value, "maxAgeInDays": 30},
                headers={"Key": abuse_key, "Accept": "application/json"}
            )
            if r.status_code == 200:
                json = r.json()
                data = json['data']
                score = data.get('abuseConfidenceScore', 'N/A')
                country = data.get('countryCode', 'N/A')
                usage = data.get('usageType', 'N/A')
                domain = data.get('domain', 'N/A')
                total_reports = data.get('totalReports', 'N/A')
                results.append(
                    f"(AbuseIPDB: {score}/100 confidence | Country: {country} | Usage: {usage} | Domain: {domain} | Reports: {total_reports})"
                )
        except Exception as e:
            results.append(f"(AbuseIPDB lookup error: {e})")
        time.sleep(5)

    # Scamalytics only for IPs
    if type_ == "ip" and scam_key and scam_name:
        try:
            url = f"https://api12.scamalytics.com/v3/{scam_name}/?key={scam_key}&ip={value}"
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                scam = data.get("scamalytics", {})
                ext = data.get("external_datasources", {})

                score = scam.get("scamalytics_score", "N/A")
                risk = scam.get("scamalytics_risk", "N/A")

                country = ext.get("ip2proxy_lite", {}).get("ip_country_code") or ext.get("ipinfo", {}).get("ip_country_code") or "N/A"

                blacklist_sources = []
                if ext.get("ipsum", {}).get("ip_blacklisted"):
                    blacklist_sources.append("IPsum")
                if ext.get("spamhaus_drop", {}).get("ip_blacklisted"):
                    blacklist_sources.append("Spamhaus")
                blacklisted = f"Yes ({', '.join(blacklist_sources)})" if blacklist_sources else "No"

                proxy_flags = []
                proxy_data = scam.get("scamalytics_proxy", {})
                if proxy_data.get("is_vpn"):
                    proxy_flags.append("VPN")
                if proxy_data.get("is_datacenter"):
                    proxy_flags.append("Datacenter")
                proxy_status = f"Yes - {', '.join(proxy_flags)}" if proxy_flags else "No"

                results.append(f"(Scamalytics: Score={score} (Risk={risk}) | Country={country} | Blacklisted={blacklisted} | Proxies={proxy_status})")
            else:
                results.append(f"(Scamalytics request failed: status {response.status_code})")
        except Exception as e:
            results.append(f"(Scamalytics lookup error: {e})")

    return results


# --- Generate the C2 Investigation report section ---
def get_c2_investigation_section():
    lines = []
    
    if c2_count.value.strip():
        lines.append(f"Count: {c2_count.value.strip()}")
    if c2_user_identity.value.strip():
        lines.append(f"User Identity: {c2_user_identity.value.strip()}")

    # Domains investigation
    domains = [d.strip() for d in c2_domains.value.strip().splitlines() if d.strip()]
    if domains:
        lines.append("\nDestination Domains:")
        for d in domains:
            defanged = defang(d)
            lines.append(f"- {defanged}")
            results = api_lookup_c2(d, "domain")
            if results:
                for res in results:
                    lines.append(f"  ↳ {res}")
            else:
                lines.append("  ↳ No data")

    # IPs investigation
    ips = [i.strip() for i in c2_ips.value.strip().splitlines() if i.strip()]
    if ips:
        lines.append("\nDestination IPs:")
        for ip in ips:
            defanged = defang(ip)
            lines.append(f"- {defanged}")
            results = api_lookup_c2(ip, "ip")
            if results:
                for res in results:
                    lines.append(f"  ↳ {res}")
            else:
                lines.append("  ↳ No data")

    if not lines:
        return ""

    return "[+] -------------------- C2 INVESTIGATION -------------------- [+]\n" + "\n".join(lines) + "\n"


## Phishing

In [73]:
# CODE
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import re
import requests
import base64
    
# --- Defanging Helper ---
def defang(text):
    text = text.replace('.', '[.]')
    text = text.replace(':', '[:]') if "://" not in text else text.replace('http', 'hxxp')
    return text

# --- Use stored keys ---
def get_api_key(source):
    return api_keys.get(source, "")

# --- Unified API Lookup Function ---
def api_lookup(value, type_, verbose=True):
    results = []
    if verbose:
        print(f"API lookup triggered for {type_}: {value}")

    # VirusTotal
    vt_key = get_api_key('VT')
    if type_ == "ip" and vt_key:
        if verbose:
            print(" → Trying VT IP scan")
        try:
            r = requests.get(f"https://www.virustotal.com/api/v3/ip_addresses/{value}",
                             headers={"x-apikey": vt_key})
            if r.status_code == 200:
                json = r.json()
                malicious = json['data']['attributes']['last_analysis_stats']['malicious']
                results.append(f"(VT results: {malicious} malicious detections)")
        except:
            pass
        time.sleep(15)

    if type_ == "url" and vt_key:
        if verbose:
            print(" → Trying VT URL scan")
        try:
            submit = requests.post("https://www.virustotal.com/api/v3/urls",
                                   headers={"x-apikey": vt_key},
                                   data={"url": value})
            url_id = base64.urlsafe_b64encode(value.encode()).decode().strip("=")
            time.sleep(15)
            r = requests.get(f"https://www.virustotal.com/api/v3/urls/{url_id}",
                             headers={"x-apikey": vt_key})
            if r.status_code == 200:
                json = r.json()
                malicious = json['data']['attributes']['last_analysis_stats']['malicious']
                results.append(f"(VT results: {malicious} malicious detections)")
            else:
                results.append(f"(VT: Status {r.status_code})")
        except Exception as e:
            print(f"VT URL scan error: {e}")

    # AbuseIPDB
    if type_ == "ip":
        abuse_key = get_api_key('AbuseIPDB')
        if verbose:
            print(" → Trying AbuseIPDB IP scan")
        if abuse_key:
            try:
                r = requests.get(
                    "https://api.abuseipdb.com/api/v2/check",
                    params={"ipAddress": value, "maxAgeInDays": 30},
                    headers={"Key": abuse_key, "Accept": "application/json"}
                )
                if r.status_code == 200:
                    json = r.json()
                    data = json['data']
                    score = data.get('abuseConfidenceScore', 'N/A')
                    country = data.get('countryCode', 'N/A')
                    usage = data.get('usageType', 'N/A')
                    domain = data.get('domain', 'N/A')
                    total_reports = data.get('totalReports', 'N/A')
                    results.append(
                        f"(AbuseIPDB: {score}/100 confidence score | "
                        f"Country: {country} | Usage: {usage} | Domain: {domain} | "
                        f"Total reports: {total_reports})"
                    )
            except Exception as e:
                if verbose:
                    print(f"AbuseIPDB request error: {e}")
            time.sleep(5)

    # Scamalytics
    scam_key = get_api_key('ScamKey')
    scam_name = get_api_key('ScamName')
    if type_ == "ip" and scam_key and scam_name:
        url = f"https://api12.scamalytics.com/v3/{scam_name}/?key={scam_key}&ip={value}"
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                scam = data.get("scamalytics", {})
                ext = data.get("external_datasources", {})

                # Score and Risk
                score = scam.get("scamalytics_score", "N/A")
                risk = scam.get("scamalytics_risk", "N/A")

                # Country
                country = ext.get("ip2proxy_lite", {}).get("ip_country_code") or ext.get("ipinfo", {}).get("ip_country_code") or "N/A"

                # Blacklist check
                blacklist_sources = []
                if ext.get("ipsum", {}).get("ip_blacklisted"):
                    blacklist_sources.append("IPsum")
                if ext.get("spamhaus_drop", {}).get("ip_blacklisted"):
                    blacklist_sources.append("Spamhaus")
                blacklisted = f"Yes ({', '.join(blacklist_sources)})" if blacklist_sources else "No"

                # Proxy check
                proxy_flags = []
                proxy_data = scam.get("scamalytics_proxy", {})
                if proxy_data.get("is_vpn"):
                    proxy_flags.append("Anonymizing VPN")
                if proxy_data.get("is_datacenter"):
                    proxy_flags.append("Datacenter")
                if proxy_flags:
                    proxy_status = f"Yes - {', '.join(proxy_flags)}"
                else:
                    proxy_status = "No"

                results.append(f"(Scamalytics: Score: {score} (Risk: {risk}) | Country: {country} | Blacklisted: {blacklisted} | Proxies: {proxy_status})")
            else:
                if verbose:
                    print(f"Scamalytics request failed with status {response.status_code}")
        except Exception as e:
            if verbose:
                print(f"Scamalytics lookup failed: {e}")

    # URLScan
    if type_ == "url":
        urlscan_key = get_api_key('URLScan')
        if verbose:
            print(" → Trying URLScan link scan")
        if urlscan_key:
            try:
                submission = requests.post("https://urlscan.io/api/v1/scan/",
                                           headers={"API-Key": urlscan_key, "Content-Type": "application/json"},
                                           json={"url": value})
                if submission.status_code == 200:
                    scan_data = submission.json()
                    result_link = scan_data.get("result")
                    uuid = scan_data.get("uuid")
                    time.sleep(15)
                    result = requests.get(f"https://urlscan.io/api/v1/result/{uuid}")
                    if result.status_code == 200:
                        data = result.json()
                        verdicts = data.get("verdicts", {}).get("overall", {})
                        score = verdicts.get("score", "unknown")
                        results.append(f"(URLScan: Score = {score}, [View Scan]({result_link}))")
                    else:
                        results.append(f"(URLScan: Submitted. [View Scan]({result_link}))")
                else:
                    results.append("(URLScan: Failed to submit URL)")
            except Exception as e:
                print(f"   URLScan error: {e}")
            time.sleep(5)

    return results


# --- Input Widgets ---
phishing_date = widgets.Text(placeholder="[Date]", layout=widgets.Layout(width='100%'))
phsender_address = widgets.Text(placeholder="[Sender email address]", layout=widgets.Layout(width='100%'))
phsender_display = widgets.Text(placeholder="[Sender display name]", layout=widgets.Layout(width='100%'))
phreturn_path = widgets.Text(placeholder="[Return Path]", layout=widgets.Layout(width='100%'))
phreply_to = widgets.Text(placeholder="[Reply-To]", layout=widgets.Layout(width='100%'))
phsender_ip = widgets.Text(placeholder="[Sender IP]", layout=widgets.Layout(width='100%'))
phfirst_hop = widgets.Text(placeholder="[Mail server IP]", layout=widgets.Layout(width='100%'))
phthreat = widgets.Text(placeholder="[Defender Threat Label]")
phemails_last_30d = widgets.Text(placeholder="[Emails from domain in the last 30 days, any similar ones?]", layout=widgets.Layout(width='100%'))
phclicks = widgets.Text(placeholder="[Number of clicks]")

# --- New Dropdown options ---
delivery_options = ["", "Dropped", "Failed", "Inbox/folder", "Junk folder", "On-prem/external", "Quarantine", "Unknown"]

# --- Dropdowns ---
spf = widgets.Dropdown(options=["", "Pass", "Fail", "Neutral", "Softfail", "None"], description="SPF:")
dkim = widgets.Dropdown(options=["", "Pass", "Fail", "Neutral", "None"], description="DKIM:")
dmarc = widgets.Dropdown(options=["", "Pass", "Fail", "None"], description="DMARC:")
original_delivery = widgets.Dropdown(options=delivery_options, description="Original:")
latest_delivery = widgets.Dropdown(options=delivery_options, description="Latest:")

# --- Text Areas ---
phlinks = widgets.Textarea(placeholder="[URLs]", layout=widgets.Layout(width='100%', height='60px'))
phemail_chain = widgets.Textarea(placeholder="[Email Chain]", layout=widgets.Layout(width='100%', height='60px'))
phrecipients = widgets.Textarea(placeholder="[Recipients]", layout=widgets.Layout(width='100%', height='60px'))
additional_info = widgets.Textarea(placeholder="[Additional information]", layout=widgets.Layout(width='100%', height='60px'))

# --- Attachments ---
attachments = []
attachment_output = widgets.Output()

def fetch_vt_for_hash(sha256):
    vt_key = get_api_key("VT")
    if not vt_key:
        return "No VT API key"
    try:
        r = requests.get(f"https://www.virustotal.com/api/v3/files/{sha256}",
                         headers={"x-apikey": vt_key})
        if r.status_code == 200:
            json = r.json()
            print(f"VT file lookup response for {sha256}:", json)
            malicious = json.get('data', {}).get('attributes', {}).get('last_analysis_stats', {}).get('malicious', 0)
            return f"{malicious} malicious detections"
        else:
            return f"VT API returned status {r.status_code}"
    except Exception as e:
        return f"Error retrieving VT: {e}"

def check_all_hashes(_):
    for (filename, sha256, vt_result) in attachments:
        sha = sha256.value.strip()
        if sha:
            vt_result.value = "Checking..."
            vt_result.value = fetch_vt_for_hash(sha)

def add_attachment(_):
    filename = widgets.Text(placeholder="[Filename]")
    sha256 = widgets.Text(placeholder="[SHA256]")
    vt_result = widgets.Label(value="")

    def on_sha256_submit(_):
        sha = sha256.value.strip()
        if sha:
            vt_result.value = fetch_vt_for_hash(sha)

    sha256.on_submit(on_sha256_submit)
    attachments.append((filename, sha256, vt_result))
    with attachment_output:
        display(widgets.VBox([
            widgets.HBox([filename, sha256]),
            vt_result
        ]))

add_attachment_btn = widgets.Button(description="Add Attachment")
add_attachment_btn.on_click(add_attachment)

check_all_hashes_btn = widgets.Button(description="Hashes Reputation")
check_all_hashes_btn.on_click(check_all_hashes)

# --- Organizing Email Chain ---
def parse_email_chain(raw_text):
    lines = [line.strip() for line in raw_text.strip().splitlines() if line.strip()]
    events = []
    for i in range(0, len(lines), 6):
        if i + 5 < len(lines):
            timeline = lines[i]
            source = lines[i+1]
            event_type = lines[i+2]
            result = lines[i+3]
            threat = lines[i+4]
            details = lines[i+5]
            events.append((timeline, source, event_type, result, threat, details))
    return events

def format_email_chain(events):
    output = ["Email Chain:"]
    for e in events:
        timeline, source, event_type, result, threat, details = e
        output.append(f"{timeline} | {source} – {event_type}")
        output.append(f"  - Result: {result}")
        output.append(f"  - Threat: {threat}")
        output.append(f"  - Details: {details}")
    return output

# --- Display UI ---
display(widgets.VBox([
    phishing_date, phsender_address, phsender_display, phreturn_path, phreply_to, phrecipients,
    phsender_ip, phfirst_hop, phlinks, phclicks,
    widgets.Label("Attachments:"),
    widgets.HBox([add_attachment_btn, check_all_hashes_btn]),
    attachment_output,
    widgets.Label("Delivery Locations:"),
    widgets.HBox([original_delivery, latest_delivery]),
    widgets.Label("Authentication Protocols:"),
    widgets.HBox([dmarc, dkim, spf]),
    phthreat, phemails_last_30d, phemail_chain, additional_info
]))

# --- Report Formatter ---
def get_phishing_section():
    lines = []
    
    def add(label, val):
        if val.strip():
            lines.append(f"{label}: {val.strip()}")

    # Email metadata
    add("Date", phishing_date.value)
    add("Sender Address", phsender_address.value)
    add("Sender Display Name", phsender_display.value)
    add("Return Path", phreturn_path.value)
    add("Reply-To", phreply_to.value)
    if phrecipients.value.strip():
        lines.append(f"Recipients: {phrecipients.value.strip()}")

    # Sender IP + First Hop with lookup results
    for label, ip in [("Sender IP", phsender_ip.value), ("First Hop", phfirst_hop.value)]:
        if ip.strip():
            defanged = defang(ip.strip())
            lines.append(f"{label}: {defanged}")
            ip_results = api_lookup(ip.strip(), "ip", verbose=False)
            if ip_results:
                for r in ip_results:
                    lines.append(f"  ↳ {r.strip('()')}")
            else:
                lines.append("  ↳ No data")

    # Links section
    if phlinks.value.strip():
        link_lines = phlinks.value.strip().splitlines()
        valid_links = [link.strip() for link in link_lines if link.strip()]
        if valid_links:
            lines.append("Links:")
            for link in valid_links:
                defanged = defang(link)
                lines.append(f"- {defanged}")
                scan_results = api_lookup(link, "url", verbose=False)
                for result in scan_results:
                    lines.append(f"  ↳ {result}")

    # Attachments section
    if attachments:
        lines.append("Attachments:")
        for i, (filename, sha256, vt_result) in enumerate(attachments, 1):
            parts = []
            if filename.value.strip():
                parts.append(f"Filename: {filename.value.strip()}")
            if sha256.value.strip():
                parts.append(f"SHA256: {sha256.value.strip()}")
            if vt_result.value.strip():
                parts.append(f"VT: {vt_result.value.strip()}")
            lines.append(f"{i}. " + " | ".join(parts))

    # Authentication protocols
    auth_protocols = []
    if spf.value: auth_protocols.append(f"SPF={spf.value}")
    if dkim.value: auth_protocols.append(f"DKIM={dkim.value}")
    if dmarc.value: auth_protocols.append(f"DMARC={dmarc.value}")
    if auth_protocols:
        lines.append("Authentication Protocols: " + ", ".join(auth_protocols))
    if original_delivery.value:
        lines.append(f"Delivery Path:\n- Original Location: {original_delivery.value}")
    if latest_delivery.value:
        if original_delivery.value:
            lines.append(f"- Latest Location: {latest_delivery.value}")
        else:
            lines.append(f"Delivery Path:\n- Latest Location: {latest_delivery.value}")
    
    # Threat & metadata
    if phthreat.value.strip():
        lines.append(f"Threat: {phthreat.value.strip()}")
    # Email chain
    if phemail_chain.value.strip():
        chain_events = parse_email_chain(phemail_chain.value)
        if chain_events:
            lines += format_email_chain(chain_events)
    
    # Additional Info
    if phclicks.value.strip() or phemails_last_30d.value.strip() or additional_info.value.strip():
        lines.append("\n>>Additional Information:")
    if phclicks.value.strip():
        lines.append(f"Clicks: {phclicks.value.strip()}")
    if phemails_last_30d.value.strip():
        lines.append(f"Emails from domain (30d): {phemails_last_30d.value.strip()}")
    if additional_info.value.strip():
        lines.append(f"{additional_info.value.strip()}")
        
    if not lines:
        return ""

    return "[+] ------------------------ PHISHING ------------------------ [+]\n>>Email Details:\n" + "\n".join(lines) + "\n"


## Queries

In [72]:
# CODE
import ipywidgets as widgets
from IPython.display import display, clear_output

# Query types options
query_types = ['SPL', 'KQL', 'Tanium Question Builder']

def create_query_block(index):
    query_type_dropdown = widgets.Dropdown(
        options=query_types,
        value=query_types[0],
        description='Type:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='250px')
    )
    query_input = widgets.Textarea(
        placeholder='QUERY USED',
        layout=widgets.Layout(width='100%', height='80px'),
        style={'description_width': 'initial'}
    )
    result_input = widgets.Textarea(
        placeholder='QUERY RESULTS',
        layout=widgets.Layout(width='100%', height='80px'),
        style={'description_width': 'initial'}
    )
    remove_btn = widgets.Button(
        description='Remove',
        button_style='danger',
        layout=widgets.Layout(width='80px')
    )

    # Container for the Remove button aligned right
    btn_box = widgets.HBox([widgets.Label(''), remove_btn], layout=widgets.Layout(justify_content='flex-end'))

    top_row = widgets.HBox([query_type_dropdown])
    query_box = widgets.VBox([top_row, query_input, result_input, btn_box])
    query_accordion = widgets.Accordion(children=[query_box])
    query_accordion.set_title(0, f"Query {index+1}")
    
    # Bundle all widgets for easy access
    query_accordion.query_type = query_type_dropdown
    query_accordion.query_text = query_input
    query_accordion.result_input = result_input
    query_accordion.remove_btn = remove_btn

    # Remove button handler
    def on_remove_clicked(b):
        # Remove this block from the main list
        query_blocks.remove(query_accordion)
        # Refresh display
        refresh_query_container()

    remove_btn.on_click(on_remove_clicked)

    return query_accordion

def refresh_query_container():
    # Re-index and update titles
    for i, block in enumerate(query_blocks):
        block.set_title(0, f"Query {i+1}")
    query_container.children = query_blocks

# Container and button to add more queries
query_blocks = []
query_container = widgets.VBox()
add_query_btn = widgets.Button(description="Add Query", button_style='info')

def add_query(b=None):
    index = len(query_blocks)
    new_block = create_query_block(index)
    query_blocks.append(new_block)
    query_container.children = query_blocks

add_query_btn.on_click(add_query)

# Initialize with one query block
add_query()

# Main queries accordion
main_queries_accordion = widgets.Accordion(children=[widgets.VBox([query_container, add_query_btn])])
main_queries_accordion.set_title(0, "Analyst Queries")

display(main_queries_accordion)


def get_queries_report():
    report_lines = []
    
    for block in query_blocks:
        qtype = block.query_type.value
        query_text = block.query_text.value.strip()
        results_text = block.result_input.value.strip()
        if not query_text and not results_text:
            continue
        
        report_lines.append(f">>{qtype}")
        if query_text:
            report_lines.append(f"{query_text}")
        if results_text:
            report_lines.append(f"{results_text}\n")
        report_lines.append("---")
    
    # Remove trailing separator if it exists
    if report_lines and report_lines[-1] == "---":
        report_lines = report_lines[:-1]

    if not report_lines:
        return ""

    return "[+] ---------------------- QUERIES USED ---------------------- [+]\n" + "\n".join(report_lines) + "\n"


Accordion(children=(VBox(children=(VBox(children=(Accordion(children=(VBox(children=(HBox(children=(Dropdown(d…

## VM Lab

In [70]:
# CODE
import ipywidgets as widgets
from IPython.display import display

def create_vm_lab_block(index):
    # Input for item being investigated
    item_input = widgets.Text(
        placeholder='Item being investigated',
        description='Item:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%')
    )
    # Textarea for file content / results
    content_input = widgets.Textarea(
        placeholder='File content / results',
        layout=widgets.Layout(width='100%', height='100px'),
        style={'description_width': 'initial'}
    )
    # Remove button
    remove_btn = widgets.Button(
        description='Remove',
        button_style='danger',
        layout=widgets.Layout(width='80px')
    )
    # Container for remove button aligned right
    btn_box = widgets.HBox([widgets.Label(''), remove_btn], layout=widgets.Layout(justify_content='flex-end'))

    # VBox holding inputs + remove button
    block_box = widgets.VBox([item_input, content_input, btn_box])

    # Accordion for collapsible display
    acc = widgets.Accordion(children=[block_box])
    acc.set_title(0, f"VM Lab Item {index + 1}")

    # Bundle widgets for access
    acc.item_input = item_input
    acc.content_input = content_input
    acc.remove_btn = remove_btn

    # Remove button handler
    def on_remove_clicked(b):
        vm_lab_blocks.remove(acc)
        refresh_vm_lab_container()

    remove_btn.on_click(on_remove_clicked)

    return acc

def refresh_vm_lab_container():
    for i, block in enumerate(vm_lab_blocks):
        block.set_title(0, f"VM Lab Item {i + 1}")
    vm_lab_container.children = vm_lab_blocks

# Container and button to add more VM lab blocks
vm_lab_blocks = []
vm_lab_container = widgets.VBox()
add_vm_lab_btn = widgets.Button(description="Add VM Lab Item", button_style='info')

def add_vm_lab_item(b=None):
    index = len(vm_lab_blocks)
    new_block = create_vm_lab_block(index)
    vm_lab_blocks.append(new_block)
    vm_lab_container.children = vm_lab_blocks

add_vm_lab_btn.on_click(add_vm_lab_item)

# Initialize with one block
add_vm_lab_item()

# Main accordion wrapping VM lab section
vm_lab_main_accordion = widgets.Accordion(children=[widgets.VBox([vm_lab_container, add_vm_lab_btn])])
vm_lab_main_accordion.set_title(0, "VM Lab Investigation")

display(vm_lab_main_accordion)


def get_vm_lab_report():
    report_lines = []

    for block in vm_lab_blocks:
        item = block.item_input.value.strip()
        content = block.content_input.value.strip()
        if not item and not content:
            continue  # skip empty blocks
        report_lines.append(f"Item: {item}")
        if content:
            report_lines.append(f"{content}\n")
        report_lines.append("---")
    
    # Remove trailing separator if it exists
    if report_lines and report_lines[-1] == "---":
        report_lines = report_lines[:-1]

    if not report_lines:
        return ""

    return "[+] ------------------------- VM LAB ------------------------- [+]\n" + "\n".join(report_lines) + "\n"


Accordion(children=(VBox(children=(VBox(children=(Accordion(children=(VBox(children=(Text(value='', descriptio…

## Conclusion

In [71]:
# CODE
import ipywidgets as widgets
from IPython.display import display

conclusion_input = widgets.Textarea(
    placeholder='Conclusion',
    layout=widgets.Layout(width='100%', height='120px'),
    style={'description_width': 'initial'},
)

display(conclusion_input)

def get_conclusion_section():
    text = conclusion_input.value.strip()
    if not text:
        return ""
    return "[+] ----------------------- CONCLUSION ----------------------- [+]\n" + text + "\n"

Textarea(value='', layout=Layout(height='120px', width='100%'), placeholder='Conclusion', style=TextStyle(desc…

## Generate Report

In [85]:
# CODE
from concurrent.futures import ThreadPoolExecutor
from ipywidgets import IntProgress
from IPython.display import display, clear_output

# Define functions for parallel API lookups
def parallel_api_lookup(values, type_):
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(lambda v: api_lookup(v, type_, verbose=False), values))
    return results

def parallel_api_lookup_c2(values, type_):
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(lambda v: api_lookup_c2(v, type_), values))
    return results

# Add progress tracking
def calculate_total_api_calls():
    domains = [d.strip() for d in c2_domains.value.strip().splitlines() if d.strip()]
    ips = [i.strip() for i in c2_ips.value.strip().splitlines() if i.strip()]
    links = [link.strip() for link in phlinks.value.strip().splitlines() if link.strip()]
    total = len(domains) + len(ips) + len(links)
    return total

def generate_final_report_with_progress(b=None):
    total_api_calls = calculate_total_api_calls()
    progress_bar = IntProgress(min=0, max=total_api_calls, description='Generating Report:')
    progress = 0

    def update_progress(increment):
        nonlocal progress
        progress += increment
        progress_bar.value = progress

    with loading_output:
        clear_output()
        display(progress_bar)

    def api_lookup_with_progress(value, type_):
        result = api_lookup(value, type_, verbose=False)
        update_progress(1)
        return result

    def api_lookup_c2_with_progress(value, type_):
        result = api_lookup_c2(value, type_)
        update_progress(1)
        return result

    report_sections = [
        get_conclusion_section(),
        get_triage_section(),
        get_c2_investigation_section(),
        get_entities_section(),
        get_phishing_section(),
        get_tanium_section(),
        get_mde_section(),
        get_queries_report(),
        get_vm_lab_report(),
    ]
    non_empty_sections = [section for section in report_sections if section.strip()]
    global report_text
    report_text = "[!] ==================== L1 INVESTIGATION ==================== [!]\n" + "\n".join(non_empty_sections) + "\n[!] ==================== L1 INVESTIGATION ==================== [!]"

    with report_output:
        clear_output(wait=True)
        display(Markdown(f"```\n{report_text}\n```"))

generate_report_button.on_click(generate_final_report_with_progress)